<a href="https://colab.research.google.com/github/abualhamd/nlp/blob/main/Question_generation_and_answering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question Generation

In [ ]:
import pickle

with open('/content/keyword_sentence_mapping.pkl', 'rb') as f:
  keyword_sentence_mapping =  pickle.load(f)

with open('/content/clean_text.pkl', 'rb') as f:
  clean_text =  pickle.load(f)

In [ ]:
!pip install --no-cache-dir transformers sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#the function to generate questions
from transformers import AutoModelWithLMHead, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap", use_fast=False)
model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")

def get_question(answer, context, max_length=64):
  input_text = "answer: %s  context: %s </s>" % (answer, context)
  features = tokenizer([input_text], return_tensors='pt')

  output = model.generate(input_ids=features['input_ids'], 
               attention_mask=features['attention_mask'],
               max_length=max_length)

  return tokenizer.decode(output[0])

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:1066: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


Downloading:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

In [ ]:
keyword_sentence_mapping[keys_list[0]][0]

'Great Northern War \n\nPeter made a temporary peace with the Ottoman Empire that allowed him to keep the captured fort of Azov, and turned his attention to Russian maritime supremacy.'

In [ ]:
get_question(keys_list[4], keyword_sentence_mapping[keys_list[4]][5])

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:220: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated"


"<pad> question: What was Peter the Great's rank in the RussoPersian War?</s>"

In [ ]:
#getting a random question
import random

def gen_question_for_dataset():
  keysLen = len(keys_list)
  keyIndx = random.randint(0, keysLen)
  sentences = keyword_sentence_mapping[keys_list[keyIndx]]
  sentenceIndx = random.randint(0, len(sentences))

  print(get_question(keys_list[keyIndx], keyword_sentence_mapping[keys_list[keyIndx]][sentenceIndx]))
  print('')
  print(keys_list[keyIndx])

In [ ]:
gen_question_for_dataset()

<pad> question: Peter's role in history was a focus of scholars and the general public in what country?</s>

russia


In [ ]:
!pip install pywsd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 26.9 MB 1.5 MB/s 
     |████████████████████████████████| 31.6 MB 1.3 MB/s 
  Created wheel for wn: filename=wn-0.0.23-py3-none-any.whl size=31792926 sha256=ef6bfdfa80671c833fcc7821d3081ecec4f0869868c9433fd02c8e29035a4067
  Stored in directory: /root/.cache/pip/wheels/ec/47/17/409766c99dd470f34c512000b90b83f34747c2c975769654d7
Successfully built wn


In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
#creating multiple-choice question
import requests
import json
import re
import random
from pywsd.similarity import max_similarity
from pywsd.lesk import adapted_lesk
from pywsd.lesk import simple_lesk
from pywsd.lesk import cosine_lesk
from nltk.corpus import wordnet as wn

# Distractors from Wordnet
def get_distractors_wordnet(syn,word):
    distractors=[]
    word= word.lower()
    orig_word = word
    if len(word.split())>0:
        word = word.replace(" ","_")
    hypernym = syn.hypernyms()
    if len(hypernym) == 0: 
        return distractors
    for item in hypernym[0].hyponyms():
        name = item.lemmas()[0].name()
        #print ("name ",name, " word",orig_word)
        if name == orig_word:
            continue
        name = name.replace("_"," ")
        name = " ".join(w.capitalize() for w in name.split())
        if name is not None and name not in distractors:
            distractors.append(name)
    return distractors

def get_wordsense(sent,word):
    word= word.lower()
    
    if len(word.split())>0:
        word = word.replace(" ","_")
    
    
    synsets = wn.synsets(word,'n')
    if synsets:
        wup = max_similarity(sent, word, 'wup', pos='n')
        adapted_lesk_output =  adapted_lesk(sent, word, pos='n')
        lowest_index = min (synsets.index(wup),synsets.index(adapted_lesk_output))
        return synsets[lowest_index]
    else:
        return None

# # Distractors from http://conceptnet.io/
def get_distractors_conceptnet(word):
    word = word.lower()
    original_word= word
    if (len(word.split())>0):
        word = word.replace(" ","_")
    distractor_list = [] 
    # url = "http://api.conceptnet.io/query?node=/c/en/%s/n&rel=/r/PartOf&start=/c/en/%s&limit=5"%(word,word)
    url = "http://api.conceptnet.io/query?node=/c/en/%s/n&rel=/r/IsA&start=/c/en/%s&limit=5"%(word,word)
    obj = requests.get(url).json()

    for edge in obj['edges']:
        link = edge['end']['term'] 

        # url2 = "http://api.conceptnet.io/query?node=%s&rel=/r/PartOf&end=%s&limit=10"%(link,link)
        url2 = "http://api.conceptnet.io/query?node=%s&rel=/r/IsA&end=%s&limit=10"%(link,link)
        obj2 = requests.get(url2).json()
        for edge in obj2['edges']:
            word2 = edge['start']['label']
            if word2 not in distractor_list and original_word.lower() not in word2.lower():
                distractor_list.append(word2)
                   
    return distractor_list

key_distractor_list = {}

for keyword in keyword_sentence_mapping:
    if len(keyword_sentence_mapping[keyword]) == 0:
      continue
    wordsense = get_wordsense(keyword_sentence_mapping[keyword][0],keyword)
    if wordsense:
        distractors = get_distractors_wordnet(wordsense,keyword)
        if len(distractors) ==0:
            distractors = get_distractors_conceptnet(keyword)
        if len(distractors) != 0:
            key_distractor_list[keyword] = distractors
    else:
        
        distractors = get_distractors_conceptnet(keyword)
        if len(distractors) != 0:
            key_distractor_list[keyword] = distractors

index = 1
print ("#############################################################################")
print ("NOTE::::::::  Since the algorithm might have errors along the way, wrong answer choices generated might not be correct for some questions. ")
print ("#############################################################################\n\n")
for each in key_distractor_list:
    # print(each, 
    # keyword_sentence_mapping[each][0])
    # print()
    sentence = keyword_sentence_mapping[each][0]
    pattern = re.compile(each, re.IGNORECASE)
    output = pattern.sub( " _______ ", sentence)
    print ("%s)"%(index),output)

    # print('hello', sentence)
    choices = [each.capitalize()] + key_distractor_list[each]
    top4choices = choices[:4]
    random.shuffle(top4choices)
    optionchoices = ['a','b','c','d']
    for idx,choice in enumerate(top4choices):
        print ("\t",optionchoices[idx],")"," ",choice)
    print ("\nMore options: ", choices[4:20],"\n\n")
    index = index + 1

Warming up PyWSD (takes ~10 secs)... took 8.520658016204834 secs.


#############################################################################
NOTE::::::::  Since the algorithm might have errors along the way, wrong answer choices generated might not be correct for some questions. 
#############################################################################


1) Great Northern War 

 _______  made a temporary peace with the Ottoman Empire that allowed him to keep the captured fort of Azov, and turned his attention to Russian maritime supremacy.
	 a )   Peter
	 b )   A soldier
	 c )   an alien
	 d )   A slave

More options:  ['mad scientist', 'unfortunate', 'manipulator', 'applicant', 'kook', 'third-rater', 'captor'] 


2) Title 
The imperial title of Peter the Great was the following:
By the grace of God, the most excellent and great sovereign emperor Pyotr Alekseevich the ruler of all the Russias: of Moscow, of Kiev, of Vladimir, of Novgorod,  _______  of Kazan,  _______  of Astrakhan and  _______  of Siberia, sovereign of Pskov, great prince of S

In [ ]:
# !pip install numpy --upgrade
# !pip install numpy --upgrade --ignore-installed

## now we try question_generator

In [ ]:
!git clone https://github.com/amontgomerie/question_generator

Cloning into 'question_generator'...
remote: Enumerating objects: 252, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 252 (delta 42), reused 40 (delta 39), pack-reused 188
Receiving objects: 100% (252/252), 114.54 KiB | 2.44 MiB/s, done.
Resolving deltas: 100% (129/129), done.


In [ ]:
%cd question_generator/
%load questiongenerator.py
from questiongenerator import QuestionGenerator
from questiongenerator import print_qa

/content/question_generator


In [ ]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
assert device == torch.device('cuda'), "Not using CUDA. Set: Runtime > Change runtime type > Hardware Accelerator: GPU"

In [ ]:
qg = QuestionGenerator()

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/121 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433M [00:00<?, ?B/s]

In [ ]:
#turning the sentences associated with each key-phrase into a context
keyword_context_mapping = []

context = ''
for element in keyword_sentence_mapping:
  for sentence in keyword_sentence_mapping[element]:
    context += sentence
  keyword_context_mapping.append({'key_phrase': element, 'context': context})
  context = ''

print(keyword_context_mapping)

[{'key_phrase': 'peter', 'context': "Great Northern War \n\nPeter made a temporary peace with the Ottoman Empire that allowed him to keep the captured fort of Azov, and turned his attention to Russian maritime supremacy.Early life \nPeter was named after the apostle, and described as a newborn as with good health, his mother's black, vaguely Tatar eyes, and a tuft of auburn hair.In the subsequent conflict, some of Peter's relatives and friends were murdered, including Matveev, and Peter witnessed some of these acts of political violence.The Streltsy made it possible for Sophia, the Miloslavskys (the clan of Ivan) and their allies to insist that Peter and Ivan be proclaimed joint Tsars, with Ivan being acclaimed as the senior.Illness and death \n\nIn the winter of , Peter, whose overall health was never robust, began having problems with his urinary tract and bladder.He had only two grandchildren: Peter II and Peter III.Traditionally, the years were reckoned from the purported creation 

In [ ]:
print(keyword_context_mapping[1])

{'key_phrase': 'russia', 'context': "After considerable success and the capture of many provinces and cities in the Caucasus and northern mainland Persia, the Safavids were forced to hand over territory to Russia, comprising Derbent, Shirvan, Gilan, Mazandaran, Baku, and Astrabad.Russia was illprepared to fight the Swedes, and their first attempt at seizing the Baltic coast ended in disaster at the Battle of Narva in .January] ), most commonly known as Peter the Great, was a monarch of Russia who modernised it and made it a European power.Swedish king Charles XII turned his attention to Russia, invading it in .Slavophiles in midth century deplored Peter's westernization of Russia.Russia had defeated what was considered to be one of the world's best militaries, and the victory overturned the view that Russia was militarily incompetent.One means of achieving this end was the introduction of taxes for long beards and robes in September .In his process to westernize Russia, he wanted membe

In [ ]:
def get_question(context):
  qa_list = qg.generate(
      context,
      num_questions=1, 
      answer_style='all'
  )
  return qa_list[0]['question']

In [ ]:
get_question(keyword_context_mapping[1]['context']) 

Generating questions...

Evaluating QA pairs...



"Where is Peter's grave?"

In [ ]:
#now we associate a question for each key-phrase
for element in keyword_context_mapping:
  question = get_question(element['context'])
  element['question'] = question

Generating questions...

Evaluating QA pairs...

Generating questions...

Evaluating QA pairs...

Generating questions...

Evaluating QA pairs...

Generating questions...

Evaluating QA pairs...

Generating questions...

Evaluating QA pairs...

Generating questions...

Evaluating QA pairs...



# Question Answering

In [ ]:
#not a very good model
model_name = "MaRiOrOsSi/t5-base-finetuned-question-answering"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelWithLMHead.from_pretrained(model_name)
index = random.randint(0, len(keyword_context_mapping) - 1)
question = keyword_context_mapping[index]['question']
context = clean_text#"42 is the answer to life, the universe and everything"
input = f"question: {question} context: {context}"
encoded_input = tokenizer([input],
                             return_tensors='pt',
                             max_length=512,
                             truncation=True)
output = model.generate(input_ids = encoded_input.input_ids,
                            attention_mask = encoded_input.attention_mask)
output = tokenizer.decode(output[0], skip_special_tokens=True)
print(question)
print(output)

Who was the first to add the title of the emperor to Peter the Great?
Alexander the Great


## using haystack



In [ ]:
%%bash

nvidia-smi

Thu Sep 22 13:54:30 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    27W /  70W |   2434MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip uninstall git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

Found existing installation: farm-haystack 1.10.0rc0
Uninstalling farm-haystack-1.10.0rc0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/farm_haystack-1.10.0rc0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/haystack/*
Proceed (Y/n)? y
  Successfully uninstalled farm-haystack-1.10.0rc0


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 30.3 MB/s eta 0:00:00


In [ ]:
%%bash

pip install --upgrade pip
pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-6x4rrvcm/farm-haystack_4d99afe49fbe4d638e5f0a962bc08d2b
  Resolved https://github.com/deepset-ai/haystack.git to commit 820742cac7c6d8426bb9d465106193cf79944686
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached transformers-4.21.2-py3-none-any.whl (4.7 MB)
  Created wheel for farm-haystack: filename=farm_haystack-1.10.0rc0-py3-none-any.whl size=681301 sha256=5eb0cfe630ec483584b347709925275b4c14eff696018ba2dde0c3bb130649e7
  Stored

  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-6x4rrvcm/farm-haystack_4d99afe49fbe4d638e5f0a962bc08d2b


In [ ]:
from haystack.utils import launch_es

launch_es()

In [ ]:
# Option 2: In Colab / No Docker environments: Start Elasticsearch from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(
    ["elasticsearch-7.9.2/bin/elasticsearch"], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1)  # as daemon
)
# wait until ES has started
! sleep 30

In [ ]:
# import os
# from haystack.document_stores import ElasticsearchDocumentStore

# # Get the host where Elasticsearch is running, default to localhost
# host = os.environ.get("ELASTICSEARCH_HOST", "localhost")
# document_store = ElasticsearchDocumentStore(host=host, username="", password="", index="document")

In [ ]:
from haystack.document_stores import ElasticsearchDocumentStore
text1 = "Python is an interpreted, high-level, general-purpose programming language. Created by Guido van Rossum and first released in 1991, Python's design philosophy emphasizes code readability with its notable use of significant whitespace."
text2 = "Princess Arya Stark is the third child and second daughter of Lord Eddard Stark and his wife, Lady Catelyn Stark. She is the sister of the incumbent Westerosi monarchs, Sansa, Queen in the North, and Brandon, King of the Andals and the First Men. After narrowly escaping the persecution of House Stark by House Lannister, Arya is trained as a Faceless Man at the House of Black and White in Braavos, using her abilities to avenge her family. Upon her return to Westeros, she exacts retribution for the Red Wedding by exterminating the Frey male line."
text3 = "Dry Cleaning are an English post-punk band who formed in South London in 2018.[3] The band is composed of vocalist Florence Shaw, guitarist Tom Dowse, bassist Lewis Maynard and drummer Nick Buxton. They are noted for their use of spoken word primarily in lieu of sung vocals, as well as their unconventional lyrics. Their musical stylings have been compared to Wire, Magazine and Joy Division.[4] The band released their debut single, 'Magic of Meghan' in 2019. Shaw wrote the song after going through a break-up and moving out of her former partner's apartment the same day that Meghan Markle and Prince Harry announced they were engaged.[5] This was followed by the release of two EPs that year: Sweet Princess in August and Boundary Road Snacks and Drinks in October. The band were included as part of the NME 100 of 2020,[6] as well as DIY magazine's Class of 2020.[7] The band signed to 4AD in late 2020 and shared a new single, 'Scratchcard Lanyard'.[8] In February 2021, the band shared details of their debut studio album, New Long Leg. They also shared the single 'Strong Feelings'.[9] The album, which was produced by John Parish, was released on 2 April 2021.[10]"

docs = [{"content": clean_text}]#, {"content": text2}, {"content": text3}

document_store = ElasticsearchDocumentStore()
document_store.write_documents(docs)

In [ ]:
# An in-memory TfidfRetriever based on Pandas dataframes
from haystack.nodes import TfidfRetriever

retriever = TfidfRetriever(document_store=document_store)

In [ ]:
from haystack.nodes import FARMReader


# Load a  local model or any of the QA models on
# Hugging Face's model hub (https://huggingface.co/models)
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

In [ ]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [ ]:
def get_answer(question):
  prediction = pipe.run(
      query= question, params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
  )
  answer = (prediction['answers'][0])
  return answer.to_dict()['answer']


In [ ]:
for element in keyword_context_mapping:
  answer = get_answer(element['question'])
  element['answer'] = answer

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 16.87 Batches/s]


In [ ]:
keyword_context_mapping[3]

{'key_phrase': 'saint petersburg',
 'context': "While the Poles fought the Swedes, Peter founded the city of Saint Petersburg in , in Ingermanland (a province of the Swedish Empire that he had captured).He is interred in Saints Peter and Paul Cathedral, Saint Petersburg, Russia.However, the formation of local elites domestically was not his main priority, and the first Russian university—Saint Petersburg State University—was founded only a year before his death, in .The Tsar retained some Finnish lands close to Saint Petersburg, which he had made his capital in .In  the construction of Peterhof, a palace near Saint Petersburg, was completed.Peter valued Catherine and married her again (this time officially) at Saint Isaac's Cathedral in Saint Petersburg on  February .He forbade the building of stone edifices outside Saint Petersburg, which he intended to become Russia's capital, so that all stonemasons could participate in the construction of the new city.Between  and , and from  to , 

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame.from_dict(keyword_context_mapping)
df

,key_phrase,context,question,answer
0,peter,Peter decided that all of the children of the nobility should have some earl...,What was the first attempt to leap out of the framework of backwardness?,seek the aid of European monarchs
1,russia,"On October , soon after peace was made with Sweden, he was officially procl...",Where is Peter's grave?,"Saints Peter and Paul Cathedral, Saint Petersburg, Russia"
2,tsar,The rebellion was easily crushed before Peter returned home from England; of...,Who was the first to add the title of the emperor to Peter the Great?,Gavrila Golovkin
3,saint petersburg,"While the Poles fought the Swedes, Peter founded the city of Saint Petersbur...",What was the capital of Russia?,St Petersburg
4,great,"The Church reform of Peter the Great therefore abolished the Patriarchy, rep...",What did Peter learn from the English techniques of citybuilding?,great effect
5,russian affairs,"Still, Peter could not acquire actual control over Russian affairs.",?,The imperial title of Peter the Great


we can see at index 5 of our dataframe there's no question just a question mark which needs more investigation

In [ ]:
df.to_csv('/content/key_phrases.csv')

In [ ]:
!cp key_phrases.csv "drive/MyDrive/datasets/"